In [2]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score ,GridSearchCV
import joblib
import os

In [ ]:
df1 = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):
    weather_df = pd.read_csv(f"../資料前處理以及第三方資料/station466990weather.csv")
    weather_df = weather_df.loc[:,["DateTime","TX01","GR01","RH01","UV01"]]
    weather_df["DateTime"] = weather_df["DateTime"].astype(str)
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].str.slice(0,8))
    weather_df["時間"] = weather_df["DateTime"].str.slice(8,10)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間"],
        right_on=["日期","時間"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
combine_weather(df1)

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,時間,時段,位置,DateTime,TX01,GR01,RH01,UV01
0,20240101063001,1,2024-01-01 06:30:00,0.0,1016.544444,17.50,86.777778,50.093333,0.002222,2024-01-01,06,30,01,2024010106,18.7,0.00,63,0.00
1,20240101064001,1,2024-01-01 06:40:00,0.0,1016.589000,17.49,86.800000,163.250000,0.016000,2024-01-01,06,40,01,2024010106,18.7,0.00,63,0.00
2,20240101065001,1,2024-01-01 06:50:00,0.0,1016.660000,17.50,86.950000,326.834000,0.046000,2024-01-01,06,50,01,2024010106,18.7,0.00,63,0.00
3,20240101070001,1,2024-01-01 07:00:00,0.0,1016.730000,17.50,86.990000,470.834000,0.083000,2024-01-01,07,00,01,2024010107,18.8,0.01,64,0.00
4,20240101071001,1,2024-01-01 07:10:00,0.0,1016.774000,17.57,86.970000,659.667000,0.139000,2024-01-01,07,10,01,2024010107,18.8,0.01,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10691,20240724174001,1,2024-07-24 17:40:00,0.0,961.297000,25.22,100.000000,91.251000,0.002000,2024-07-24,17,40,01,2024072417,27.0,0.02,88,0.04
10692,20240724175001,1,2024-07-24 17:50:00,0.0,961.056000,24.91,100.000000,53.999000,0.000000,2024-07-24,17,50,01,2024072417,27.0,0.02,88,0.04
10693,20240724180001,1,2024-07-24 18:00:00,0.0,960.907000,24.76,100.000000,37.248000,0.000000,2024-07-24,18,00,01,2024072418,27.1,0.01,92,0.01
10694,20240724181001,1,2024-07-24 18:10:00,0.0,959.856000,24.97,100.000000,40.332000,0.000000,2024-07-24,18,10,01,2024072418,27.1,0.01,92,0.01


In [ ]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1))

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,時間,時段,位置,DateTime,TX01,GR01,RH01,UV01
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,09,00,01,2024010109,18.8,0.19,64,0.00
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,09,10,01,2024010109,18.8,0.19,64,0.00
17,20240101092001,1,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,09,20,01,2024010109,18.8,0.19,64,0.00
18,20240101093001,1,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,09,30,01,2024010109,18.8,0.19,64,0.00
19,20240101094001,1,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,09,40,01,2024010109,18.8,0.19,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10683,20240724162001,1,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,16,20,01,2024072416,25.6,0.07,95,0.22
10684,20240724163001,1,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,16,30,01,2024072416,25.6,0.07,95,0.22
10685,20240724164001,1,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,16,40,01,2024072416,25.6,0.07,95,0.22
10686,20240724165001,1,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,16,50,01,2024072416,25.6,0.07,95,0.22


In [ ]:
# d = {}
# for i in range(1,18):
#     l1 = "0" + str(i) if i<10 else str(i)
#     df1 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
#     df2 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
#     df1 = addcolumns(df1)
#     df1 = combine_weather(df1)
#     df1 = selecttime(df1)
#     df2 = addcolumns(df2)
#     df2 = combine_weather(df2)
#     df2 = selecttime(df2)
#     df2["對應日期"] =  df2["日期"] - pd.Timedelta(days=1)
#     merge_df = pd.merge(
#             df1,
#             df2,
#             left_on=['日期', '時間', '時段'],    # 左表連接的鍵
#             right_on=['對應日期', '時間', '時段'], # 右表連接的鍵
#             how='inner',
#             suffixes=["_x","_y"]                         # 左表有x 代表我們要用左觀測站的資料來預測右觀測站的power
#         )
#     X = merge_df.loc[:,["WindSpeed(m/s)_x","Pressure(hpa)_x", "Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x",
#                             "TX01_x","GR01_x","RH01_x","UV01_x","Power(mW)_x"]].reset_index(drop=True)
#     Y = merge_df.loc[:,["Power(mW)_y"]].reset_index(drop=True)

    

In [ ]:
d = {}
for i in range(1,18):
    l1 = "0" + str(i) if i<10 else str(i)
    df1 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
    # df2 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
    df2 = df1.copy()
    df1 = addcolumns(df1)
    df1 = combine_weather(df1)
    df1 = selecttime(df1)
    df2 = addcolumns(df2)
    df2 = combine_weather(df2)
    df2 = selecttime(df2)
    df2["對應日期"] =  df2["日期"] - pd.Timedelta(days=1)
    merge_df = pd.merge(
        df1,
        df2,
        left_on=['日期', '時間', '時段'],    # 左表連接的鍵
        right_on=['對應日期', '時間', '時段'], # 右表連接的鍵
        how='inner',
        suffixes=["_x","_y"]                         # 左表有x 代表我們要用左觀測站的資料來預測右觀測站的power
    )

    X = merge_df.loc[:,["WindSpeed(m/s)_x","Pressure(hpa)_x", "Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x",
                        "TX01_x","GR01_x","RH01_x","UV01_x","Power(mW)_x"]].reset_index(drop=True)
    Y = merge_df.loc[:,["Power(mW)_y"]].reset_index(drop=True)
    Y = Y.values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

    # model = RandomForestRegressor(
              
    # )
    # scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    # MAE = -sum(scores)
    # model.fit(X_train, y_train)
    param_grid = {
        'n_estimators': [100, 250, 400],
        'max_depth': [10, 20, 30],
        'max_features': ['sqrt', 'log2']
    }

    # 使用 GridSearchCV 搜索最佳參數
    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid=param_grid,
        scoring='neg_mean_squared_error',  # 使用 MSE 作為評估標準
        cv=5,
    )
    grid_search.fit(X_train, y_train)
    
    # 儲存最佳模型和 MSE
    best_model = grid_search.best_estimator_
    scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)

    MAE = -sum(scores)  # 取負數以還原 MSE

    model_file_name = f'{l1}.pkl'  
    model_folder_name = "repeat_train_lastday_gridsearch_usealldata"         #將模型相關資訊儲存在這個檔案下
    model_file_path = os.path.join("./model_each_location",model_folder_name,model_file_name)
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(best_model, model_file_path)
    print(model_file_name ," MAE:",MAE )
    print(f"模型已保存至: {model_file_path}")
    
    if l1 not in d:
        d[model_file_name] = MAE

01.pkl  MAE: 792.4590311074239
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gridsearch_usealldata\01.pkl
02.pkl  MAE: 952.2292019385019
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gridsearch_usealldata\02.pkl
03.pkl  MAE: 1050.9047825920854
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gridsearch_usealldata\03.pkl
04.pkl  MAE: 748.6029629334728
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gridsearch_usealldata\04.pkl
05.pkl  MAE: 968.4640312062534
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gridsearch_usealldata\05.pkl
06.pkl  MAE: 985.6469878829058
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_lastday_gr

# 紀錄每個model的MAE並且存成json檔案

In [ ]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"./model_each_location/{model_folder_name}/d_lastday.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)